# Sample code

## Tokyo Datum (EPSG:4301) to JGD2000 (EPSG:4612)

In [1]:
from pathlib import Path
from jgdtrans import Point, loads

# load TKY2JGD.par
s = Path("TKY2JGD.par").read_text(encoding='cp932')
tf = loads(s, format="TKY2JGD")

# points to transform
points = [
    Point(36.10377479, 140.087855041),
]

# transformation
[tf.forward(*point) for point in points]

[Point(latitude=36.106966279935016, longitude=140.08457686562787, altitude=0.0)]

## JGD2000 (EPSG:4612) to JGD2011 (EPSG:6668)

In [2]:
from pathlib import Path
from jgdtrans import Point, loads, ParameterNotFoundError

# load touhokutaiheiyouoki2011.par
s = Path("touhokutaiheiyouoki2011.par").read_text(encoding='cp932')
tf = loads(s, format="PatchJGD")

# points to transform
points = [
    Point(36.10377479, 140.087855041),
]

# use origin if it is locate in the area where the parameter not support)
sucess = []
error = []
for point in points:
    try:
        res = tf.forward(*point)
    except ParameterNotFoundError:
        error.append(point)
    else:
        sucess.append(res)

print(f"{len(error)} / {len(points)} out-of-range")
print(sucess)
print(error)

0 / 1 out-of-range
[Point(latitude=36.10377396066534, longitude=140.0878624973557, altitude=0.0)]
[]


## Effect of the 2011 Tohoku Earthquake (東日本大震災)

In [3]:
import math
from pathlib import Path

import folium
from folium.plugins import MarkerCluster

from jgdtrans import Point, loads

In [4]:
def arrow_icon(
    intensity: int,
    angle: int,
    color: str = "black",       
) -> folium.DivIcon:
    """Returns arrow folium.DivIcon"""
    size = 2 * max(1.3 * math.ceil(intensity * max(abs(math.cos(angle)), abs(math.sin(angle)))), 25)
    return folium.DivIcon(
        html="""<div>
        <svg
        xmlns="http://www.w3.org/2000/svg" version="1.1"viewBox="-{origin} -{origin} {size} {size}"stroke="{color}" fill="{color}">
        <g transform="rotate({angle})">
        <path d="M {tip} 0 l -12 3 l 0 -6 Z" />
        <path d="M 0 0 h {e}" stroke-width="3" />
        </g>
        </svg>
        </div>""".format(
            angle=angle,
            strength=intensity,
            tip=intensity,
            e=intensity - 12,
            size=size,
            origin=size // 2,
            color=color
        ),
        icon_anchor=(size//2, size//2),
        icon_size=(size, size)
    )

In [5]:
m = folium.Map(
    location=[35.690921, 139.700258],
    zoom_start=6,
    # GIAJ tiles
    tiles='https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png',
    attr='GIAJ',
    name='GIAJ'
)

# add Cartodb
m.add_child(folium.TileLayer(tiles='Cartodb Positron', attr='Cartodb', name='Cartodb'))

pass

In [6]:
# 
# latitude, longitude
# 
path = Path("touhokutaiheiyouoki2011.par")
tf = loads(path.read_text(encoding='cp932'), format="PatchJGD")

# clustering markers
horizontal = MarkerCluster(
    name='horizontal',
    # use the first marker (very naive)
    icon_create_function = """function(cluster) {
const markers = cluster.getAllChildMarkers();
return markers[0].getIcon();
}""",
    max_cluster_radius = 80,
)

maximum = max(math.hypot(parameter.latitude, parameter.longitude) for parameter in tf.parameter.values())
for code, parameter in tf.parameter.items():
    point = Point.from_meshcode(code)
    node = point.to_mesh_node(tf.unit)
    # filter to decrease markers
    if not (node.latitude.third in (0, ) and node.longitude.third  in (0, )):
        continue
    
    intensity = math.hypot(parameter.latitude, parameter.longitude)
    angle = math.degrees(math.atan2(-parameter.latitude, parameter.longitude))
    
    # yellow to red
    saturation = 90 * (0.5 - intensity / maximum)

    folium.Marker(
        # location
        [point.latitude, point.longitude],
        # icon
        icon=arrow_icon(
            500 * intensity,
            angle,
            color=f"hsl({saturation} 100% 50%)",
        ),
        # popup
        popup="<p>intensity: {:.1f} [1e6 deg]</p>".format(1000 * 1000 * intensity / 3600)
    ).add_to(horizontal)

In [7]:
# 
# altitude
# 
p = Path("touhokutaiheiyouoki2011_h.par")
tf = loads(p.read_text(encoding='cp932'), format="PatchJGD_H")

# clustering markers
vertical = MarkerCluster(
    name='vertical',
    # use the first marker (very naive)
    icon_create_function = """function(cluster) {
const markers = cluster.getAllChildMarkers();
return markers[0].getIcon();
}""",
    max_cluster_radius = 60,
)

for code, parameter in tf.parameter.items():
    point = Point.from_meshcode(code)
    node = point.to_mesh_node(tf.unit)

    # filter to decrease markers
    if not (node.latitude.third in (0, ) and node.longitude.third  in (0, )):
        continue

    intensity = abs(parameter.altitude)
    angle = -90 if 0 < parameter.altitude else 90
    color = "red" if 0 < parameter.altitude else "blue"

    folium.Marker(
        # location
        [point.latitude, point.longitude],
        # icon
        icon=arrow_icon(
            100 * intensity,
            angle,
            color=color
        ),
        # popup
        popup="<p>intensity: {:.1f} [cm]</p>".format(100 * intensity)
    ).add_to(vertical)

In [8]:
m.add_child(horizontal)
m.add_child(vertical)
m.add_child(folium.LayerControl())